# Predicción de aprobación curso de matematicas mediante árboles de decisión.


---------
* Juan Esteban Ortiz - 2410227-3743
* Juan David Olaya - 202410206-3743
* Pablo Esteban Becerra - 202243506 - 3743
* Fernando Cardona Giraldo - 202241381-3743
* Sara Yineth Suarez Reyes - 202241923-3743

Para la experimentación con técnicas de machine learning se empleará un conjunto de datos 
compuesto por 1044 personas que participaron en un estudio orientado a predecir si aprueban un curso 
de matemáticas. La información recopilada incluye variables como edad, sexo, ocupación de la madre 
y del padre, tiempo de estudio semanal, acceso a internet y otros datos relevantes. Cada estudiante 
está descrito mediante los 17 atributos. La variable dependiente es 
approved, que toma el valor 1 cuando el estudiante aprueba el curso y 0 en caso contrario. En este 
informe se desarrollan modelos para predecir dicha variable a partir de los demás atributos, abordando 
el problema como una tarea de clasificación. 

*   Se cargan las bibliotecas que se usarán

In [765]:
import sklearn
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

* Lectura de datos

In [766]:
dataFrame1 =pd.read_csv('student_performance.csv')
dataFrame1

,sex,age,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,internet,romantic,goout,Walc,health,approved
0,F,18,GT3,A,4,4,at_home,teacher,2,2,0,no,no,4,1,3,0
1,F,17,GT3,T,1,1,at_home,other,1,2,0,yes,no,3,1,3,0
2,F,15,LE3,T,1,1,at_home,other,1,2,3,yes,no,2,3,3,1
3,F,15,GT3,T,4,2,health,services,1,3,0,yes,yes,2,1,5,1
4,F,16,GT3,T,3,3,other,other,1,2,0,no,no,2,2,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,F,19,GT3,T,2,3,services,other,1,3,1,yes,no,2,2,5,1
1040,F,18,LE3,T,3,1,teacher,services,1,2,0,yes,no,4,1,1,1
1041,F,18,GT3,T,1,1,other,other,2,2,0,no,no,1,1,5,0
1042,M,17,LE3,T,3,1,services,services,2,1,0,yes,no,5,4,2,1


In [767]:
dataFrame1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sex         1044 non-null   object
 1   age         1044 non-null   int64 
 2   famsize     1044 non-null   object
 3   Pstatus     1044 non-null   object
 4   Medu        1044 non-null   int64 
 5   Fedu        1044 non-null   int64 
 6   Mjob        1044 non-null   object
 7   Fjob        1044 non-null   object
 8   traveltime  1044 non-null   int64 
 9   studytime   1044 non-null   int64 
 10  failures    1044 non-null   int64 
 11  internet    1044 non-null   object
 12  romantic    1044 non-null   object
 13  goout       1044 non-null   int64 
 14  Walc        1044 non-null   int64 
 15  health      1044 non-null   int64 
 16  approved    1044 non-null   int64 
dtypes: int64(10), object(7)
memory usage: 138.8+ KB


# Se separan los datos en entrenamiento y prueba
80% para entrenar y 20% para pruebas


In [768]:
train_data,test_data=sklearn.model_selection.train_test_split(dataFrame1, test_size=0.2)
print(len(train_data),len(test_data))

835 209


# Pipeline para los atributos categóricos

In [769]:
# 7 atributos categóricos
cat_attribs = ['sex','famsize','Pstatus','Mjob','Fjob','internet','romantic']

cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output=False))
    ])

# Pipeline para los atributos numéricos

In [770]:
# 9 atributos numéricos
num_attribs = ['age','Medu','Fedu','traveltime','studytime','failures','goout','Walc','health']

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

# Pipeline completo

In [771]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [772]:
X_train = train_data.drop("approved", axis=1)
y_train = train_data["approved"]

In [773]:
X_test = test_data.drop("approved", axis=1)
y_test = test_data["approved"]

In [774]:
X_train_transform = full_pipeline.fit_transform(X_train)
X_test_transform = full_pipeline.transform(X_test)

print(X_train_transform.shape, X_test_transform.shape)


(835, 29) (209, 29)


# Entrenamiento de árboles de decisión

## Usando Criterio gini

In [811]:
modelo1 = tree.DecisionTreeClassifier(max_depth=2, criterion='gini')
modelo1.fit(X_train_transform, y_train)

DecisionTreeClassifier(max_depth=2)

In [776]:
modelo2 = tree.DecisionTreeClassifier(max_depth=4, criterion='gini')
modelo2.fit(X_train_transform, y_train)

DecisionTreeClassifier(max_depth=4)

In [777]:
modelo3 = tree.DecisionTreeClassifier(max_depth=6, criterion='gini')
modelo3.fit(X_train_transform, y_train)

DecisionTreeClassifier(max_depth=6)

In [778]:
modelo4 = tree.DecisionTreeClassifier(max_depth=8, criterion='gini')
modelo4.fit(X_train_transform, y_train)

DecisionTreeClassifier(max_depth=8)

In [779]:
modelo5 = tree.DecisionTreeClassifier(max_depth=10, criterion='gini')
modelo5.fit(X_train_transform, y_train)

DecisionTreeClassifier(max_depth=10)

### Uso de los modelos entrenados

In [780]:
predicciones1 = modelo1.predict(X_test_transform)
predicciones2 = modelo2.predict(X_test_transform)
predicciones3 = modelo3.predict(X_test_transform)
predicciones4 = modelo4.predict(X_test_transform)
predicciones5 = modelo5.predict(X_test_transform)

In [781]:
accuracy1 = accuracy_score(y_test, predicciones1)
accuracy2 = accuracy_score(y_test, predicciones2)
accuracy3 = accuracy_score(y_test, predicciones3)
accuracy4 = accuracy_score(y_test, predicciones4)
accuracy5 = accuracy_score(y_test, predicciones5)

In [782]:
resultados = {
    'Técnica': ['Modelo 1','Modelo 2','Modelo 3','Modelo 4','Modelo 5'],
    'max_depth': [2, 4, 6, 8, 10],
    'Accuracy': [accuracy1,accuracy2,accuracy3,accuracy4,accuracy5]
}

tabla_resultados = pd.DataFrame(resultados)
tabla_resultados = tabla_resultados.round(4)
tabla_resultados

,Técnica,max_depth,Accuracy
0,Modelo 1,2,0.8086
1,Modelo 2,4,0.8373
2,Modelo 3,6,0.7273
3,Modelo 4,8,0.7273
4,Modelo 5,10,0.7129


---------

## Usando Criterio Entropy

In [783]:
modelo1_entropy = tree.DecisionTreeClassifier(max_depth=2, criterion='entropy')
modelo1_entropy.fit(X_train_transform, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=2)

In [784]:
modelo2_entropy = tree.DecisionTreeClassifier(max_depth=4, criterion='entropy')
modelo2_entropy.fit(X_train_transform, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [785]:
modelo3_entropy = tree.DecisionTreeClassifier(max_depth=6, criterion='entropy')
modelo3_entropy.fit(X_train_transform, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6)

In [786]:
modelo4_entropy = tree.DecisionTreeClassifier(max_depth=8, criterion='entropy')
modelo4_entropy.fit(X_train_transform, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=8)

In [787]:
modelo5_entropy = tree.DecisionTreeClassifier(max_depth=10, criterion='entropy')
modelo5_entropy.fit(X_train_transform, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

### Uso de los modelos entrenados

In [788]:
predicciones1_entropy = modelo1_entropy.predict(X_test_transform)
predicciones2_entropy = modelo2_entropy.predict(X_test_transform)
predicciones3_entropy = modelo3_entropy.predict(X_test_transform)
predicciones4_entropy = modelo4_entropy.predict(X_test_transform)
predicciones5_entropy = modelo5_entropy.predict(X_test_transform)

In [789]:
accuracy1_1 = accuracy_score(y_test, predicciones1_entropy)
accuracy1_2 = accuracy_score(y_test, predicciones2_entropy)
accuracy1_3 = accuracy_score(y_test, predicciones3_entropy)
accuracy1_4 = accuracy_score(y_test, predicciones4_entropy)
accuracy1_5 = accuracy_score(y_test, predicciones5_entropy)

In [790]:
resultados = {
    'Técnica': ['Modelo 1','Modelo 2','Modelo 3','Modelo 4','Modelo 5'],
    'max_depth': [2, 4, 6, 8, 10],
    'Accuracy': [accuracy1_1,accuracy1_2,accuracy1_3,accuracy1_4,accuracy1_5]
 
}
tabla_resultados = pd.DataFrame(resultados)
tabla_resultados = tabla_resultados.round(4)
tabla_resultados

,Técnica,max_depth,Accuracy
0,Modelo 1,2,0.8086
1,Modelo 2,4,0.8373
2,Modelo 3,6,0.8182
3,Modelo 4,8,0.7656
4,Modelo 5,10,0.7081


## Hiperparámetros que permiten obtener el árbol con mayor *accuracy*

**Respuesta:**  

El mayor *accuracy* obtenido entre todos los modelos fue **0.8373**, correspondiente al árbol entrenado con:

- max_depth = 4
- criterion = gini y entropy. Ambos produjeron el mismo

Se puede observar que la estructura del árbol resultante es similar tanto para el criterio gini como para entropy, esto sucede porque ambos criterios están encontrando exactamente las mismas divisiones óptimas en el árbol.


-----------

In [803]:
# Hiperparametros con mayor accuracy 
bestCriterion = "gini" # o Entropy :b
bestMax_depth = 4

In [804]:
m1 = tree.DecisionTreeClassifier(max_depth=bestMax_depth, criterion=bestCriterion, min_samples_split=2)
m1.fit(X_train_transform, y_train)
acc1 = accuracy_score(y_test, m1.predict(X_test_transform))

In [806]:
m2 = tree.DecisionTreeClassifier(max_depth=bestMax_depth, criterion=bestCriterion, min_samples_split=10)
m2.fit(X_train_transform, y_train)
acc2 = accuracy_score(y_test, m2.predict(X_test_transform))

In [807]:
m3 = tree.DecisionTreeClassifier(max_depth=bestMax_depth, criterion=bestCriterion, min_samples_split=20)
m3.fit(X_train_transform, y_train)
acc3 = accuracy_score(y_test, m3.predict(X_test_transform))

In [808]:
resultados = {
    'min_samples_split': ['2','10','20'],
    'Accuracy': [acc1,acc2,acc3] 
}

tabla_resultados = pd.DataFrame(resultados)
tabla_resultados = tabla_resultados.round(4)
tabla_resultados

,min_samples_split,Accuracy
0,2,0.8373
1,10,0.8373
2,20,0.8373


Se utilizó el hiperparámetro min_samples_split y se realizaron tres variaciones usando los hiperparámetros que proporcionaron el mayor accuracy, con el fin de observar cómo podría cambiar el rendimiento del modelo. Sin embargo, se puede observar que el accuracy se mantiene igual con diferentes valores de min_samples_split porque max_depth = 4 limita el crecimiento del árbol, haciendo que todos los valores probados resulten equivalentes.